In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# mpl.rcParams['figure.dpi'] = 300
# mpl.rc('font', size=12)

from numba import njit

# Voter model with mutation

In [4]:
@njit
def init_grid(K):
    '''Initialises a K by K grid with numbers drawn randomly from (0, 1)'''
    return np.random.rand(K, K)

In [5]:
@njit
def get_8_neighbors(i, j, K):
    '''Finds all 8 neighbors of a site in a K by K grid'''
    neighbors = []
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            # Skip the current site
            if di == 0 and dj == 0:
                continue
            # Apply periodic boundary conditions
            neighbor_i = int((i + di) % K)  
            neighbor_j = int((j + dj) % K)
            neighbors.append((neighbor_i, neighbor_j))
    return neighbors

@njit
def get_4_neighbors(i, j, K):
    '''Finds upper, lower, left and right neighbor of a site in a K by K grid'''
    neighbors = []
    # Up, down, right, left
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  
    for di, dj in directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % K)
        neighbor_j = int((j + dj) % K)
        neighbors.append((neighbor_i, neighbor_j))
    return neighbors

In [6]:
import random

@njit
def voter_model(grid_0, alpha, n_iters):
    '''Run experiment with the voter model
    Inputs:
    grid_0 (numpy array): Initial grid
    alpha (float): Value of alpha parameter
    n_iters (int): Number of times that the entire grid is updated
    
    Returns:
    cur_grid (numpy array): Grid after n_iter iterations
    num_species (list): Contains amount of different species at each tenth iteration
    '''
    # Create list to store number of unique species
    num_species = []
    cur_grid = np.copy(grid_0)
    
    height, width = cur_grid.shape
        
    # Update grid n_iter times
    for i in range(n_iters):
        x = np.random.choice(np.arange(0, width))
        y = np.random.choice(np.arange(0, height))
    
        if random.random() < alpha:
            cur_grid[x, y] = random.random()
        # Set species in cell to that in one of its 4 neighbors with equal probability
        else:
            neighbors = get_4_neighbors(x, y, L)
            neighbor_idx = np.random.choice(len(neighbors))
            new_type = cur_grid[neighbors[neighbor_idx]]
            cur_grid[x, y] = new_type
        # Save amount of species every tenth iteration
        if i % (n_iters//100) == 0:
            num_species.append(len(np.unique(cur_grid)))
    return cur_grid, num_species